# PPMROB - Laborübung
Dr. Dietmar Schreiner

---

# Einführung in pyTorch

## Tensoren

* Ein Tensor ist ein n-dimensionales Array (in der Mathematik n-dimensionaler Tensor)
* In pyTorch ist ein Tensor eine Instanz der Klasse *torch.tensor*.

### Rank, Axes und Shape

#### Rank
Der Rang (rank) eines Tensors gibt die Anzahl der Dimensionen des Tensors an. D.h. der Rang gibt an, wieviele Indizes zur Adressierung von Elementen des Tensors benötigt werden.

__Beispiel__: Ein rank-2 Tensor
* ist eine Matrix
* ist ein 2d-Array
* ist ein 2d-Tensor

#### Axes
Eine Achse ist eine spezifische Dimension des Tensors.Für jede Achse existiert ein Index zur Adressierung der Elemente des Tensors. Ein rank-2 Tensor hat also 2 Dimensionen und damit 2 Achsen und somit 2 Indizes. 

Achsen besitzen eine spezifische Länge (length), wodurch der Gültigkeitsbereich der Indizes wie auch der Speicherverbrauch des Tensors bekannt ist.



In [1]:
# python matrix (allgemein)

td = [
    [1,2,3],
    [4,5,6],
    [7,8,9]
]

In [2]:
td

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

In [3]:
td[0]

[1, 2, 3]

In [4]:
td[0][2]

3

#### Shape

Die Shape eines Tensors gibt die konkrete Ausformung eines Tensors wieder. Sie enthält die vollständige Information zu den Achsen, dem Rang und damit den Indizes. 

In pyTorch werden die Begriffe __size__ und __shape__ für Tensoren gleichbedeutend verwendet.


In [5]:
import torch

In [6]:
# create pyTorch tensor from python list
t=torch.tensor(td)
t

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [7]:
type(t)

torch.Tensor

In [8]:
t.shape

torch.Size([3, 3])

Der Rank eines Tensors entspricht der Länge seiner Shape.(__Hinweis__: python list!)

In [9]:
len(t.shape)

2

##### Reshaping eines Tensors

Reshaping verändert die Anordnung/Gruppierung von Daten, jedoch nicht die Daten selbst.

__Beispiel__: 6 Datenpunkte ~ Werte

Shape 6 x 1: (*Rohdaten*)
   - number
   - scalar
   - array
   - vector
   - 2d-array
   - matrix
   
Shape 2 x 3: (*Einteilung nach Informatik/Mathematik*)
   - number, array, 2d-array
   - scalar, vector, matrix

Shape 3 x 2: (*Map von Informatik nach Mathematik*)
   - number, scalar
   - array, vector
   - 2d-array, matrix
        

In [10]:
# reshape t from 3x3 to 1x9 (flatten)
t.reshape(1,9)

tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9]])

In [11]:
t.reshape(1,9).shape

torch.Size([1, 9])

##### Beispiel: Input eines CNN

Ein CNN erhält typischerweise einen 4-dimensionalen Tensor als Input.

```python
len([?,?,?,?])==4
```

Die Zordnung der Achsen dabei ist __[B,C,H,W]__ mit
* B: Batch
* C: Channel
* H: Height 
* W: Width

Ein Batch aus 3 28x28 Graustufen Bildern wird folglich in einem Tensor mit der Shape \[3,1,28,28\] an des CNN übergeben.


### pyTorch spezifische Tensor-Attribute

#### Data Type
Das Attribut *dtype* spezifiziert den zugrundeliegenden Datentyp der Elemente des Tensors.

In [12]:
print(t.dtype)

torch.int64


#### Device
Das Attribut *device* spezifiziert, in welcher PU (CPU oder GPU) die Daten des Tensors allokiert sind. pyTorch ermöglicht die Verwendung mehrerer PUs gleichen Typs. Diese werden durch eindeutige Indizes unterschieden (z.B. 'cuda:2').

In [13]:
print(t.device)

cpu


In [14]:
device = torch.device(t.device)
device

device(type='cpu')

#### Stride
Der Stride eines Tensors gibt an, wieviele Speicherplätze weiter das nächste Element im Tensor abgelegt ist. Der Stride kann nicht kleiner als die Größe eines Elements des Tensors sein. Entspricht der Stride genau der Elementgröße, so spricht man von einem fortlaufenden (*contiguous*) Tensor.

Das Layout eines pyTorch Tensors kann mittel des Attributs *layout* ermittelt werden.

In [15]:
print(t.layout)

torch.strided


### Konstruktion eines Tensors aus Daten



In [16]:
import numpy as np

data = np.array([1,2,3])
type(data)

numpy.ndarray

#### Konstruktion mittels Konstruktor der Klasse Tensor

In [17]:
torch.Tensor(data) # class constructor

tensor([1., 2., 3.])

Der Konstruktor der Tensor Klasse erzeugt einen Tensor aus Elementen des Typs *float*, genauer gesagt von Elementen des Typs *default_dtype*.

In [18]:
# dtype used by Tensor() constructor
torch.get_default_dtype()

torch.float32

#### Konstruktion mittels Factory Funktionen

Die in pyTorch vorhandenen Factory Functions zur Erzeugung eines Tensors inferieren den Datentyp der Tensorelemente aus den Aufrufparametertypen.

In [19]:
torch.tensor(data) # factory function, *

tensor([1, 2, 3])

In [20]:
torch.tensor(np.array([1.,2.,3.]))

tensor([1., 2., 3.], dtype=torch.float64)

In [21]:
torch.as_tensor(data) # factory function, zero memory-copy, *

tensor([1, 2, 3])

In [22]:
torch.from_numpy(data) # factory function, zero memory-copy

tensor([1, 2, 3])

Der Datentyp der Tensorelemente kann für alle Factory Funktionen auch händisch festgelegt werden.

In [23]:
torch.tensor(np.array([1,2,3]), dtype=torch.float64)

tensor([1., 2., 3.], dtype=torch.float64)

#### Memory: Sharing vs. Copying
* Der __Klassenkonstruktor__ und die Factory Function __*torch.tensor()*__ realisieren eine __Copy-by-Value__ Semantik
* Die Factory Function __*torch.as_tensor()*__ und die Factory Function __*torch.from_numpy()*__ realisieren eine __Copy-by-Reference__ Semantik 

In [24]:
data

array([1, 2, 3])

In [25]:
t1=torch.Tensor(data)
t2=torch.tensor(data)
t3=torch.as_tensor(data)
t4=torch.from_numpy(data)

Nun werden die Ausgangsdaten verändert...

In [26]:
data[0]=0
data[1]=0
data[2]=0
data

array([0, 0, 0])

Die beiden Tensoren, die mit Copy-by-Value Semantik erzeugt wurden behalten ihre Originalwerte.

In [27]:
print(t1)
print(t2)

tensor([1., 2., 3.])
tensor([1, 2, 3])


Die beiden Tensoren, die mit Copy-by-Reference Semantik erzeugt wurden weisen die aktualisierten Werte aus.

In [28]:
print(t3)
print(t4)

tensor([0, 0, 0])
tensor([0, 0, 0])


__Anmerkungen__:
1. *numpy.ndarray* Objekte werden immer auf der CPU allokiert. Werden Tensoroperationen aber auf der GPU ausgeführt, muss die *torch.as_tensor()* Funktion den CPU-Speicher in die GPU kopieren.
2. Das Memory-Sharing von *torch.as_tensor()* funktioniert nicht für built-in Datentypen von Python.
3. Die potentielle Performance-Steigerung durch *torch.as_tensor()* wird kaum relevant, wenn der Tensor nur einmalig geladen wird.

### Konstruktion eines Tensors ohne Daten

In [29]:
torch.eye(3) # Identitätsmatrx

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [30]:
torch.zeros(3,3)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [31]:
torch.ones(3,3)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [32]:
torch.rand(3,3)

tensor([[0.1045, 0.9464, 0.1383],
        [0.2302, 0.7624, 0.6178],
        [0.7514, 0.0135, 0.5159]])

### Erzeugung eines Tensors auf der CPU oder GPU

pyTorch ermöglicht auf sehr einfache Weise zu entscheiden, ob Berechnungen auf der CPU oder der GPU ausgeführt werden sollen.

__Achtung__: Alle Operanden einer Berechnung müssen sich auf der selben PU befinden!

### Deklaration der Variable auf der CPU


In [33]:
t=torch.tensor([1,2,3,4,5])
t

tensor([1, 2, 3, 4, 5])

### Zuweisung der tensor variable zu einer GPU

In [34]:
#t=t.cuda() # not avail
#t

## Tensor Operationen

Die wichtigsten Tensor Operationen sind
* Zugriffsoperationen
* Reshaping Operationen
* Elementweise Operationen
* Reduktionsoperationen

### Zugriffsoperationen

Zugriffsoperationen sind all jene Funktionen, die den direkten Zugriff auf Elemente des Tensors ermöglichen. Diese sind zum Beispiel der Index Operator.

### Reshaping Operationen

In [35]:
t = torch.tensor([
    [1,1,1,1],
    [2,2,2,2],
    [3,3,3,3]
], dtype=torch.float32)

In [36]:
# shape
t.size()

torch.Size([3, 4])

In [37]:
# shape
t.shape

torch.Size([3, 4])

In [38]:
# rank
len(t.shape)

2

In [39]:
# element count
torch.tensor(t.shape).prod()

tensor(12)

In [40]:
# element.count
t.numel()

12

#### reshape()
Die Reshape Operation verändert die Shape des Tensors, nicht aber die zugrundeliegenden Daten. Deshalb muss __das Produkt aller Achsenlängen immer gleich der Anzahl der Elemente des Tensors__ sein.

##### Unveränderter Rang

In [41]:
t.reshape(1,12)

tensor([[1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.]])

In [42]:
t.reshape(2,6)

tensor([[1., 1., 1., 1., 2., 2.],
        [2., 2., 3., 3., 3., 3.]])

In [43]:
t.reshape(3,4)

tensor([[1., 1., 1., 1.],
        [2., 2., 2., 2.],
        [3., 3., 3., 3.]])

In [44]:
t.reshape(4,3)

tensor([[1., 1., 1.],
        [1., 2., 2.],
        [2., 2., 3.],
        [3., 3., 3.]])

In [45]:
t.reshape(6,2)

tensor([[1., 1.],
        [1., 1.],
        [2., 2.],
        [2., 2.],
        [3., 3.],
        [3., 3.]])

In [46]:
t.reshape(12,1)

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [2.],
        [2.],
        [2.],
        [2.],
        [3.],
        [3.],
        [3.],
        [3.]])

##### Veränderter Rang

Auch wenn der Rang des Tensors verändert wird, muss das __Produkt der Achsenlängen wieder gleich der Anzahl der Elemente__ sein.

In [47]:
t.reshape(2,2,3)

tensor([[[1., 1., 1.],
         [1., 2., 2.]],

        [[2., 2., 3.],
         [3., 3., 3.]]])

#### squeeze() und unsqueeze()

* __*squeeze()*__ entfernt alle Achsen mit einer Länge von 1
* __*unsqueeze()*__ fügt Achsen mit einer Länge von 1 hinzu


In [48]:
# print the original tensor and it's size
print(t.reshape(1,12))
print(t.reshape(1,12).shape)

tensor([[1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.]])
torch.Size([1, 12])


In [49]:
# print the squeezed tensor and it's size
print(t.reshape(1,12).squeeze())
print(t.reshape(1,12).squeeze().shape)

tensor([1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.])
torch.Size([12])


In [50]:
# print the unsqueezed squeezed tensor and it's size
print(t.reshape(1,12).squeeze().unsqueeze(dim=0))
print(t.reshape(1,12).squeeze().unsqueeze(dim=0).shape)

tensor([[1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.]])
torch.Size([1, 12])


##### Beispiel: Flatten

Die Flatten Operation wird beispielsweise am Übergang von einem Convolutional Layer zu einem Dense Layer benötigt.

In [51]:
def flatten(t):
    t = t.reshape(1,-1) # -1 denotes 'find out yourself!'
    t = t.squeeze()
    return t

In [52]:
flatten(t)

tensor([1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.])

#### Concatenation

__*cat()*__ verbindet zwei Tensoren. Die Shape des durch die Operation entstehenden neuen Tensors hängt dabei von der Shape der beiden Operanden ab.


In [53]:
# define operands a and b
a=torch.tensor([
    [1,2],
    [3,4]
])
b=torch.tensor([
    [5,6],
    [7,8]
])

Zeilenweise Kombination:

In [54]:
torch.cat((a,b), dim=0)

tensor([[1, 2],
        [3, 4],
        [5, 6],
        [7, 8]])

Spaltenweise Kombination:

In [55]:
torch.cat((a,b), dim=1)

tensor([[1, 2, 5, 6],
        [3, 4, 7, 8]])

__*stack()*__ verbindet mehrere Tensoren entlang einer neuen Achse.

##### Beispiel: Erstellen eines Batches für ein CNN

In [56]:
# define operands t1, t2, t3
t1=torch.tensor([
    [1,1,1,1],
    [1,1,1,1],
    [1,1,1,1],
    [1,1,1,1]
])
t2=torch.tensor([
    [2,2,2,2],
    [2,2,2,2],
    [2,2,2,2],
    [2,2,2,2]
])
t3=torch.tensor([
    [3,3,3,3],
    [3,3,3,3],
    [3,3,3,3],
    [3,3,3,3]
])

Verbindung entlang einer neuen Achse:

In [57]:
ts=torch.stack((t1,t2,t3))
ts.shape

torch.Size([3, 4, 4])

In [58]:
ts

tensor([[[1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1]],

        [[2, 2, 2, 2],
         [2, 2, 2, 2],
         [2, 2, 2, 2],
         [2, 2, 2, 2]],

        [[3, 3, 3, 3],
         [3, 3, 3, 3],
         [3, 3, 3, 3],
         [3, 3, 3, 3]]])

Typische CNNs erwarten als Input einen 4-dimensionalen Tensor (\[B,C,H,W\]). Der Tensor ts muss daher mit einer reshape Operation angepasst werden.

In [59]:
ts=ts.reshape(3,1,4,4) # add a new Axis of length 1 to encapsulate the image data
ts

tensor([[[[1, 1, 1, 1],
          [1, 1, 1, 1],
          [1, 1, 1, 1],
          [1, 1, 1, 1]]],


        [[[2, 2, 2, 2],
          [2, 2, 2, 2],
          [2, 2, 2, 2],
          [2, 2, 2, 2]]],


        [[[3, 3, 3, 3],
          [3, 3, 3, 3],
          [3, 3, 3, 3],
          [3, 3, 3, 3]]]])

Der Zugriff auf die einzelnen Komponenten des Tensors erfolgt nun wie folgt:

In [60]:
# first image
ts[0]

tensor([[[1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1]]])

In [61]:
# first channel of the first image
ts[0][0]

tensor([[1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]])

In [62]:
# first row of pixels in the first channel of the first image
ts[0][0][0]

tensor([1, 1, 1, 1])

In [63]:
# first pixel value of the first row of pixels in the first channel of the first image
ts[0][0][0][0]

tensor(1)

Um eine Flatten Operation auf die einzelnen Bilder im Batch anzuwenden, ohne den gesamten Batch ebenfalls zu linearisieren aknn die __*flatten()*__ Operation von pyTorch verwendet werden: 

In [64]:
ts.flatten(start_dim=1)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]])

In [65]:
ts.flatten(start_dim=1).shape

torch.Size([3, 16])

### Elementweise Operationen

Elementweise Operationen sind Operationen zwischen zwei Tensoren, die korrespondierende Elemente dieser Tensoren verknüpfen. Zwei Elemente in den beiden Tensoren werden als korrespondierend bezeichnet, wenn sie die selbe Position innerhalb des jeweiligen Tensors innehaben. Die Position wird durch die Indizes bestimmt, über die auf das Element zugegriffen werden kann.

Elementweise Operationen setzen Operanden der selben Shape voraus.

In [66]:
t1 = torch.tensor([
    [1,2],
    [3,4]
], dtype=torch.float32)
t2 = torch.tensor([
    [9,8],
    [7,6]
], dtype=torch.float32)

#### Arithmetische Operationen am Beispiel Addition

In [67]:
t1+t2

tensor([[10., 10.],
        [10., 10.]])

##### Addition mit einem Skalar

__Problem__: Skalare sind Tensoren mit Rang 0 und haben daher keine Shape!

__Lösung__: Skalare werden für elementweise Operationen mit Tensoren mittels einer __*Broadcast*__ Funktion in einen Tensor mit Shape des Operandentensors umgewandelt. Danach kann eine elementweise Addition erfolgen.

In [68]:
# this works thanks to broadcast
t1+2

tensor([[3., 4.],
        [5., 6.]])

#### Broadcasting

In [69]:
# convert scalar 1 to tensor
np.broadcast_to(2,t1.shape)

array([[2, 2],
       [2, 2]])

In [70]:
# this addition is equivalent to the one above
t1+torch.tensor(
    np.broadcast_to(2,t1.shape),
    dtype=torch.float32
)

tensor([[3., 4.],
        [5., 6.]])

__Beispiel für zwei Tensoren unterschiedlicher Shape__

In [71]:
t1 = torch.tensor([
    [1,1],
    [1,1]
], dtype=torch.float32)
t2 = torch.tensor([
    [2,4]
], dtype=torch.float32)

In [72]:
t1.shape

torch.Size([2, 2])

In [73]:
t2.shape

torch.Size([1, 2])

Kann t1+t2 berechnet werden?

In [74]:
# shpw result of broadcasting
np.broadcast_to(t2.numpy(),t1.shape)

array([[2., 4.],
       [2., 4.]], dtype=float32)

In [75]:
t1 + t2

tensor([[3., 5.],
        [3., 5.]])

#### Vergleichsoperationen

Vergleichsoperationen sind ebenfalls elementweise Operationen. Sie liefern als Ergebnis einen Tensor der Shape des Operandentensors zurück, der Boolsche Werte für den elementweisen Vergleich enthält. 

In [76]:
t = torch.tensor([
    [0,5,7],
    [6,0,7],
    [0,8,0]
])

In [77]:
t.eq(0)

tensor([[ True, False, False],
        [False,  True, False],
        [ True, False,  True]])

In [78]:
t.ge(0)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

In [79]:
t.gt(0)

tensor([[False,  True,  True],
        [ True, False,  True],
        [False,  True, False]])

In [80]:
t.lt(0)

tensor([[False, False, False],
        [False, False, False],
        [False, False, False]])

In [81]:
t.le(7)

tensor([[ True,  True,  True],
        [ True,  True,  True],
        [ True, False,  True]])

#### Funktionsoperatoren

Auch Funktionen können elementweise auf einen Tensor angewendet werden.

In [82]:
t = torch.tensor([
    [1, 0, 0],
    [0,-1, 0],
    [0, 0, 1]
])

In [83]:
t.neg()

tensor([[-1,  0,  0],
        [ 0,  1,  0],
        [ 0,  0, -1]])

In [84]:
t.abs()

tensor([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

### Reduktionsoperationen

Reduktionsoperationen verringern die Anzahl der Elemente innerhalb eines Tensors.

In [85]:
t = torch.tensor([
    [0,1,0],
    [2,0,2],
    [0,3,0]
], dtype=torch.float32)

#### sum()
__*sum()*__ summiert alle Elemente eines Tensors auf.

In [86]:
t.sum()

tensor(8.)

Um anstelle eines rank-0 Tensors ein Skalar als Ergebnis zu erhalten kann die item() Funktion des Tensors aufgerufen werden.

In [87]:
t.sum().item()

8.0

#### prod()
__*prod()*__ multipliziert alle Elemente eines Tensors.

In [88]:
t.prod()

tensor(0.)

#### mean()
__*mean()*__ berechnet den Mittelwert aller Elemente eines Tensors.

In [89]:
t.mean()

tensor(0.8889)

#### std()
__*std()*__ berechnet die Standardabweichung aller Elemente eines Tensors.

In [90]:
t.std()

tensor(1.1667)

#### Allgemeine Reduktion (partiell)

In [91]:
t = torch.tensor([
    [1,1,1,1],
    [2,2,2,2],
    [3,3,3,3]
], dtype=torch.float32)

In [92]:
# reduce along first axis
t.sum(dim=0)

tensor([6., 6., 6., 6.])

Diese Reduktion entspricht:

In [93]:
t[0]+t[1]+t[2]

tensor([6., 6., 6., 6.])

In [94]:
# reduce along second axis
t.sum(dim=1)

tensor([ 4.,  8., 12.])

#### Argmax Reduktion
Die Argmax Reduktion reduziert einen Tensor auf die Position seines größten Elements.

In [95]:
t = torch.tensor([
    [1,0,0,2],
    [0,3,3,0],
    [4,0,0,5]
], dtype=torch.float32)

In [96]:
# get the max value of the tensor
t.max()

tensor(5.)

In [97]:
# get position of max value in flattened tensor
t.argmax()

tensor(11)

In [98]:
t.flatten()

tensor([1., 0., 0., 2., 0., 3., 3., 0., 4., 0., 0., 5.])

In [99]:
# positions of max value within each column (dim 0)
t.argmax(dim=0)

tensor([2, 1, 1, 2])

In [100]:
t.max(dim=1)

torch.return_types.max(
values=tensor([2., 3., 5.]),
indices=tensor([3, 1, 3]))

In [101]:
# position of max value within each row (dim 1)
t.argmax(dim=1)

tensor([3, 1, 3])